The Benjamini Yekutieli (BY) procedure is a multiple testing procedure that can be used to control the accumulation in type 1 errors when comparing multiple hypothesis at the same time.

In the tsfresh filtering the BY procedure is used to decide which features to use and which to keep. 

The method is based on a line, the so called rejection line, that is compared to the sequence of ordered p-values. In this notebook, we will visualize that rejection line.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import defaults, extract_features
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

matplotlib.rcParams["figure.figsize"] = [16, 6]
matplotlib.rcParams["font.size"] = 14
matplotlib.style.use('seaborn-darkgrid')

## Parameter setting

In [ ]:
FDR_LEVEL = defaults.FDR_LEVEL
HYPOTHESES_INDEPENDENT = defaults.HYPOTHESES_INDEPENDENT

## Load robot data

In [ ]:
download_robot_execution_failures()
df, y = load_robot_execution_failures()
df.head()

## Extract Features

In [ ]:
X = extract_features(df, 
                     column_id='id', column_sort='time',
                     default_fc_parameters=ComprehensiveFCParameters(),
                     impute_function=impute)

In [ ]:
# drop constant features
print(X.shape)
X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
print(X.shape)

## Calculate p-values and Benjamini-Yekutieli Procedure

tsfresh has implemented two different feature significance tests, the Mann-Whitney-U test and the Kolmogorov-Smirnov test. In the following, both of them are being illustrated to show a scientific report of the feature selection process and to give a comparison of the differences of both methods.

### Mann-Whitney-U
Run significance test with Mann-Whitney-U test. Returns the p-values of the features and whether they are rejected or not.

In [ ]:
df_pvalues_mann = calculate_relevance_table(X, y, fdr_level=FDR_LEVEL, test_for_binary_target_real_feature='mann')

In [ ]:
print("# total \t", len(df_pvalues_mann))
print("# relevant \t", (df_pvalues_mann["relevant"] == True).sum())
print("# irrelevant \t", (df_pvalues_mann["relevant"] == False).sum(),
      "( # constant", (df_pvalues_mann["type"] == "const").sum(), ")")

In [ ]:
df_pvalues_mann.head()

### Kolmogorov-Smirnov
Run significance test with Kolmogorov-Smirnov test. Returns the p-values of the features and whether they are rejected or not.

In [ ]:
df_pvalues_smir = calculate_relevance_table(X, y, fdr_level=FDR_LEVEL, test_for_binary_target_real_feature='smir')

In [ ]:
print("# total \t", len(df_pvalues_smir))
print("# relevant \t", (df_pvalues_smir["relevant"] == True).sum())
print("# irrelevant \t", (df_pvalues_smir["relevant"] == False).sum(),
      "( # constant", (df_pvalues_smir["type"] == "const").sum(), ")")

In [ ]:
df_pvalues_smir.head()

## Calculate rejection line
With the rejection line it is determined whether a feature is relevant or irrelevant.

In [ ]:
def calc_rejection_line(df_pvalues, hypothesis_independent, fdr_level):
    
    m = len(df_pvalues.loc[~(df_pvalues.type == "const")])
    K = list(range(1, m + 1))
    
    if hypothesis_independent:
        C = [1] * m
    else:
        C = [sum([1.0 / k for k in K])] * m

    return [fdr_level * k / m * 1.0 / c for k, c in zip(K, C)]  

### Mann-Whitney-U

In [ ]:
rejection_line_mann = calc_rejection_line(df_pvalues_mann, HYPOTHESES_INDEPENDENT, FDR_LEVEL)

### Kolmogorov-Smirnov

In [ ]:
rejection_line_smir = calc_rejection_line(df_pvalues_smir, HYPOTHESES_INDEPENDENT, FDR_LEVEL)

## Plot ordered p-values and rejection line

In the plot, the p-values are ordered from low to high. Constant features (green points) are always irrelevant but are not considered for calculating the rejection line (red line).

For nice plotting, the p-values are divided in the three groups relevant, irrelevant and constant (which are also irrelevant).

### Mann-Whitney-U

In [ ]:
df_pvalues_mann.index = pd.Series(range(0, len(df_pvalues_mann.index)))

df_pvalues_mann.p_value.where(df_pvalues_mann.relevant)\
    .plot(style=".", label="relevant features")

df_pvalues_mann.p_value.where(~df_pvalues_mann.relevant & (df_pvalues_mann.type != "const"))\
    .plot(style=".", label="irrelevant features")

df_pvalues_mann.p_value.fillna(1).where(df_pvalues_mann.type == "const")\
    .plot(style=".", label="irrelevant (constant) features")

plt.plot(rejection_line_mann, label="rejection line (FDR = " + str(FDR_LEVEL) + ")")
plt.xlabel("Feature #")
plt.ylabel("p-value")
plt.title("Mann-Whitney-U")
plt.legend()
plt.plot()

### Kolmogorov-Smirnov

In [ ]:
df_pvalues_smir.index = pd.Series(range(0, len(df_pvalues_smir.index)))

df_pvalues_smir.p_value.where(df_pvalues_smir.relevant)\
    .plot(style=".", label="relevant features")

df_pvalues_smir.p_value.where(~df_pvalues_smir.relevant & (df_pvalues_smir.type != "const"))\
    .plot(style=".", label="irrelevant features")

df_pvalues_smir.p_value.fillna(1).where(df_pvalues_smir.type == "const")\
    .plot(style=".", label="irrelevant (constant) features")

plt.plot(rejection_line_smir, label="rejection line (FDR = " + str(FDR_LEVEL) + ")")
plt.xlabel("Feature #")
plt.ylabel("p-value")
plt.title("Kolmogorov-Smirnov")
plt.legend()
plt.plot()

## Plot zoomed ordered p-values and rejection line
Since the intersection of the ordered p-values and the rejection line is not clearly visible, a zoomed plot is provided.

### Mann-Whitney-U

In [ ]:
last_rejected_index = (df_pvalues_mann["relevant"] == True).sum() - 1
margin = 20
a = max(last_rejected_index - margin, 0)
b = min(last_rejected_index + margin, len(df_pvalues_mann) - 1)

df_pvalues_mann[a:b].p_value.where(df_pvalues_mann[a:b].relevant)\
    .plot(style=".", label="relevant features")
df_pvalues_mann[a:b].p_value.where(~df_pvalues_mann[a:b].relevant)\
    .plot(style=".", label="irrelevant features")
plt.plot(np.arange(a, b), rejection_line_mann[a:b], label="rejection line (FDR = " + str(FDR_LEVEL) + ")")
plt.xlabel("Feature #")
plt.ylabel("p-value")
plt.title("Mann-Whitney-U")
plt.legend()
plt.plot()

### Kolmogorov-Smirnov

In [ ]:
last_rejected_index = (df_pvalues_smir["relevant"] == True).sum() - 1
margin = 20
a = max(last_rejected_index - margin, 0)
b = min(last_rejected_index + margin, len(df_pvalues_smir) - 1)

df_pvalues_smir[a:b].p_value.where(df_pvalues_smir[a:b].relevant)\
    .plot(style=".", label="relevant features")
df_pvalues_smir[a:b].p_value.where(~df_pvalues_smir[a:b].relevant)\
    .plot(style=".", label="irrelevant features")
plt.plot(np.arange(a, b), rejection_line_smir[a:b], label="rejection line (FDR = " + str(FDR_LEVEL) + ")")
plt.xlabel("Feature #")
plt.ylabel("p-value")
plt.title("Kolmogorov-Smirnov")
plt.legend()
plt.plot()